In [44]:
# Dependencies
import requests
import xml.etree.ElementTree as ET # to parse XML
import csv
from config import API_KEY

import numpy as np
import pandas as pd

# API source : GreatSchools
https://www.greatschools.org/api/docs/school-profile/
response is returned in XML format.

* XML format explanation : https://www.w3schools.com/xml/xml_tree.asp
* cf. parsing example : https://www.datacamp.com/community/tutorials/python-xml-elementtree

## testing with CA

In [62]:
# URL prep
URL = 'https://api.greatschools.org/schools/CA/"*"?'
queryURL = URL + 'key=' + API_KEY
# print(queryURL)

In [63]:
# request
response = requests.get(queryURL)
response.content

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><schools><school><gsId>17662</gsId><name> \tAlliance Patti And Peter Neuwirth Leadership Academy </name><type>charter</type><gradeRange>9-12</gradeRange><enrollment>630</enrollment><gsRating>7</gsRating><parentRating>5</parentRating><city>Los Angeles</city><state>CA</state><districtId>286</districtId><district>Los Angeles Unified School District</district><districtNCESId>0622710</districtNCESId><address>4610 South Main Street, \nLos Angeles, CA  90037</address><phone>(213) 342-2874</phone><fax>(213) 943-4931</fax><website>http://collegeready5.org</website><ncesId>062271011632</ncesId><lat>34.001568</lat><lon>-118.27383</lon><overviewLink>https://www.greatschools.org/california/los-angeles/17662--\tAlliance-Patti-And-Peter-Neuwirth-Leadership-Academy-/?utm_source=GSAPI&amp;utm_medium=referral</overviewLink><ratingsLink>https://www.greatschools.org/california/los-angeles/17662--\tAlliance-Patti-And-Peter-Neuwirth-Leadership-Academy

In [101]:
# parse
tree = ET.fromstring(response.content)

df = pd.DataFrame()
schoolProfile={}

for school in tree:
    for child in school:
#         print(child.tag, child.text)
        schoolProfile[str(child.tag)] = str(child.text)
    df = df.append(schoolProfile, ignore_index=True)

print(df.columns)
print(len(df))
print(df.dtypes)
df.head()

Index(['address', 'city', 'district', 'districtId', 'districtNCESId',
       'enrollment', 'fax', 'gradeRange', 'gsId', 'gsRating', 'lat', 'lon',
       'name', 'ncesId', 'overviewLink', 'parentRating', 'phone',
       'ratingsLink', 'reviewsLink', 'schoolStatsLink', 'state', 'type',
       'website'],
      dtype='object')
200
address            object
city               object
district           object
districtId         object
districtNCESId     object
enrollment         object
fax                object
gradeRange         object
gsId               object
gsRating           object
lat                object
lon                object
name               object
ncesId             object
overviewLink       object
parentRating       object
phone              object
ratingsLink        object
reviewsLink        object
schoolStatsLink    object
state              object
type               object
website            object
dtype: object


,address,city,district,districtId,districtNCESId,enrollment,fax,gradeRange,gsId,gsRating,...,ncesId,overviewLink,parentRating,phone,ratingsLink,reviewsLink,schoolStatsLink,state,type,website
0,"4610 South Main Street, \nLos Angeles, CA 90037",Los Angeles,Los Angeles Unified School District,286,0622710,630,(213) 943-4931,9-12,17662,7,...,062271011632,https://www.greatschools.org/california/los-an...,5,(213) 342-2874,https://www.greatschools.org/california/los-an...,https://www.greatschools.org/california/los-an...,https://www.greatschools.org/california/los-an...,CA,charter,http://collegeready5.org
1,"9753 Ellington Lane, \nTruckee, CA 96161",Truckee,Los Angeles Unified School District,286,0622710,25,None,9-12,32348,7,...,None,https://www.greatschools.org/california/trucke...,5,(530) 536-5333,https://www.greatschools.org/california/trucke...,https://www.greatschools.org/california/trucke...,https://www.greatschools.org/california/trucke...,CA,private,None
2,"939 Michigan Avenue, \nBeaumont, CA 92223",Beaumont,Beaumont Unified School District,515,0604290,100,None,K-12,31844,2,...,060429013779,https://www.greatschools.org/california/beaumo...,5,(951) 769-8424,https://www.greatschools.org/california/beaumo...,https://www.greatschools.org/california/beaumo...,https://www.greatschools.org/california/beaumo...,CA,public,http://glenview-beaumont-ca.schoolloop.com/21s...
3,"4138 Lakeside Drive, \nRichmond, CA 94806",Richmond,Beaumont Unified School District,515,0604290,29,(510) 262-1540,K-12,13424,2,...,K9300854,https://www.greatschools.org/california/richmo...,4,(510) 262-1500,https://www.greatschools.org/california/richmo...,https://www.greatschools.org/california/richmo...,https://www.greatschools.org/california/richmo...,CA,private,http://www.calautism.org/programs/abc-school/
4,"4138 Lakeside Drive, \nRichmond, CA 94806",Richmond,Beaumont Unified School District,515,0604290,37,None,4-12,24131,2,...,A0500119,https://www.greatschools.org/california/richmo...,4,(510) 262-1500,https://www.greatschools.org/california/richmo...,https://www.greatschools.org/california/richmo...,https://www.greatschools.org/california/richmo...,CA,private,None


In [103]:
# select/sort columns and drop unnecessary columns.
df_selected = df[['gsId','name','type','gradeRange','enrollment','parentRating','gsRating',
         'district', 'districtId', 'districtNCESId','ncesId',
         'address','city','state','lat','lon','phone']]

In [107]:
# convert to numeric
try:
    df_selected['gsId'] = pd.to_numeric(df_selected['gsId'])
    df_selected['enrollment'] = pd.to_numeric(df_selected['enrollment'])
    df_selected['parentRating'] = pd.to_numeric(df_selected['parentRating'])
    df_selected['gsRating'] = pd.to_numeric(df_selected['gsRating'])
    df_selected['districtId'] = pd.to_numeric(df_selected['districtId'])
    df_selected['districtNCESId'] = pd.to_numeric(df_selected['districtNCESId'])
    df_selected['ncesId'] = pd.to_numeric(df_selected['ncesId'])
    df_selected['lat'] = pd.to_numeric(df_selected['lat'])
    df_selected['lon'] = pd.to_numeric(df_selected['lon'])
except (ValueError):
    print("Unable to parse/convert... skipping.")

Unable to parse... skipping.


C:\Users\soyou\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\soyou\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\soyou\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [108]:
# sort by 'gsID'
df_sorted=df_selected.sort_values(["gsId"])
df_sorted.head()

,gsId,name,type,gradeRange,enrollment,parentRating,gsRating,district,districtId,districtNCESId,ncesId,address,city,state,lat,lon,phone
110,402,Acalanes High School,public,9-12,1423,4,10,Acalanes Union High School District,39,601650,060165000032,"1200 Pleasant Hill Road, \nLafayette, CA 94549",Lafayette,CA,37.904312,-122.09906,(925) 280-3970
153,791,Addams Elementary School,public,K-6,824,4,3,Fresno Unified School District,99,614550,061455001703,"2117 West Mckinley Avenue, \nFresno, CA 93728",Fresno,CA,36.764687,-119.83641,(559) 457-2510
62,954,Abraham Lincoln Middle School,public,7-8,994,5,4,Selma Unified School District,115,636270,063627001989,"1239 Nelson Boulevard, \nSelma, CA 93662",Selma,CA,36.58507,-119.60203,(559) 898-6600
160,1180,Adelante High School,public,9-12,3,4,6,Reef-sunset Unified School District,197,632270,063227002047,"701 General Petroleum Street, \nKettleman City...",Kettleman City,CA,36.00824,-119.96231,(559) 386-9081
22,1300,A. M. Thomas Middle School,public,6-8,170,5,1,Lost Hills Union Elementary School District,221,622950,062295007772,"20979 Lobos Court, \nLost Hills, CA 93249",Lost Hills,CA,35.61527,-119.69955,(661) 797-2626


In [95]:
# drop duplicates, keep one row
df_sorted.drop_duplicates(subset='gsId', keep='last', inplace=True)
print(len(df_sorted))

200


In [96]:
df_clean = df_sorted.dropna()
print(len(df_clean))

200


In [111]:
df_sorted.to_csv('GreatSchools_CA.csv',index=False)

<hr></hr>

## testing to collect all states' data

In [121]:
# URL prep
URL = 'https://api.greatschools.org/schools/'
states=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA',
#         'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD',
#         'MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
#         'NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
#         'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY'
       ]
# print(len(states))

queryURL = f"{URL}{state}/'*'?key={API_KEY}"
# print(queryURL)

In [122]:
df_all = pd.DataFrame()
schoolProfile_all={}

for state in states:
    queryURL = f'{URL}{state}/"*"?key={API_KEY}'
    response = requests.get(queryURL)
#     print(queryURL)
    tree = ET.fromstring(response.content)
    print(tree)

    for school in tree:
        for child in school:
            print(child.tag, child.text)
            schoolProfile_all[str(child.tag)] = str(child.text)
        df_all = df_all.append(schoolProfile_all, ignore_index=True)

print(df_all.columns)
print(len(df_all))
print(df_all.dtypes)
df_all.head()
    

<Element 'schools' at 0x000002DBEA6E1228>
gsId 3708
name 100 BLK Men of Greater Mobile's Phoenix Program
type private
gradeRange 6-12
city Prichard
state AL
address 838 W Main St, 
Prichard, AL  36610
phone None
fax None
website None
ncesId A0900001
lat 30.736015
lon -88.09117
overviewLink https://www.greatschools.org/alabama/prichard/3708-100-BLK-Men-Of-Greater-Mobile's-Phoenix-Program/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/alabama/prichard/3708-100-BLK-Men-Of-Greater-Mobile's-Phoenix-Program/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/alabama/prichard/3708-100-BLK-Men-Of-Greater-Mobile's-Phoenix-Program/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/alabama/prichard/3708-100-BLK-Men-Of-Greater-Mobile's-Phoenix-Program/?utm_source=GSAPI&utm_medium=referral
gsId 468
name A M Windham Elementary School
type public
gradeRange K-6
enrollment 539
gsRating 5
parentRating 3
c

overviewLink https://www.greatschools.org/alabama/mobile/48-Alabama-School-Of-Mathematics-And-Science/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/alabama/mobile/48-Alabama-School-Of-Mathematics-And-Science/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/alabama/mobile/48-Alabama-School-Of-Mathematics-And-Science/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/alabama/mobile/48-Alabama-School-Of-Mathematics-And-Science/?utm_source=GSAPI&utm_medium=referral
gsId 2043
name Alabama Waldorf School
type private
gradeRange PK-8
enrollment 140
parentRating 4
city Birmingham
state AL
address 5901 Crestwood Blvd, Birmingham, AL 35212, 
Birmingham, AL  35222
phone (205) 592-0541
fax None
website http://www.alabamawaldorf.org/
ncesId A9703769
lat 33.5219
lon -86.75617
overviewLink https://www.greatschools.org/alabama/birmingham/2043-Alabama-Waldorf-School/?utm_source=GSAPI&utm_medium=referr

gsId 2189
name Allentown Elementary School
type public
gradeRange K-5
enrollment 825
gsRating 8
parentRating 4
city Semmes
state AL
districtId 91
district Mobile County School District
districtNCESId 0102370
address 10330 Howells Ferry Rd, 
Semmes, AL  36575
phone (251) 221-1000
fax (251) 221-1003
website http://allentown.mce.schoolinsites.com
ncesId 010237000572
lat 30.746435
lon -88.28619
overviewLink https://www.greatschools.org/alabama/semmes/2189-Allentown-Elementary-School/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/alabama/semmes/2189-Allentown-Elementary-School/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/alabama/semmes/2189-Allentown-Elementary-School/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/alabama/semmes/2189-Allentown-Elementary-School/?utm_source=GSAPI&utm_medium=referral
gsId 290
name Allgood Alt School
type public
gradeRange 2-12
parentRating 5
city Oneo

gsId 2267
name Apostolic Church of Promise Christian School
type private
gradeRange PK-1
city Birmingham
state AL
address 1133 Tuscaloosa Avenue SW, 
Birmingham, AL  35211
phone (205) 785-8200
fax None
website None
ncesId A0500004
lat 33.49493
lon -86.85075
overviewLink https://www.greatschools.org/alabama/birmingham/2267-Apostolic-Church-Of-Promise-Christian-School/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/alabama/birmingham/2267-Apostolic-Church-Of-Promise-Christian-School/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/alabama/birmingham/2267-Apostolic-Church-Of-Promise-Christian-School/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/alabama/birmingham/2267-Apostolic-Church-Of-Promise-Christian-School/?utm_source=GSAPI&utm_medium=referral
gsId 281
name Appalachian School
type public
gradeRange K-12
enrollment 582
gsRating 4
parentRating 3
city Oneonta
state AL
districtId 24

gsId 1328
name Ashville Middle School
type public
gradeRange 5-9
enrollment 344
gsRating 6
parentRating 4
city Ashville
state AL
districtId 116
district St Clair County School District
districtNCESId 0103062
address 22331 Hwy 231, 
Ashville, AL  35953
phone (205) 594-7044
fax (205) 594-2241
website http://www.sccboe.org
ncesId 010306200322
lat 33.771477
lon -86.282265
overviewLink https://www.greatschools.org/alabama/ashville/1328-Ashville-Middle-School/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/alabama/ashville/1328-Ashville-Middle-School/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/alabama/ashville/1328-Ashville-Middle-School/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/alabama/ashville/1328-Ashville-Middle-School/?utm_source=GSAPI&utm_medium=referral
gsId 1588
name Athens Bible School
type private
gradeRange K-12
enrollment 330
parentRating 4
city Athens
state AL
addre

...

gsId 150
name Auburn Elementary School
type public
gradeRange PK-5
enrollment 770
gsRating 5
parentRating 4
city Auburn
state GA
districtId 9
district Barrow County School District
districtNCESId 1300290
address 1334 6th Avenue, 
Auburn, GA  30011
phone (770) 963-7887
fax (770) 963-2923
website http://www.barrow.k12.ga.us./aes/
ncesId 130029000178
lat 34.00979
lon -83.82418
overviewLink https://www.greatschools.org/georgia/auburn/150-Auburn-Elementary-School/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/georgia/auburn/150-Auburn-Elementary-School/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/georgia/auburn/150-Auburn-Elementary-School/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/georgia/auburn/150-Auburn-Elementary-School/?utm_source=GSAPI&utm_medium=referral
gsId 1978
name Augusta Christian Schools
type private
gradeRange PK-12
enrollment 543
parentRating 4
city Martinez
sta

gsId 3945
name Avery Elementary School
type public
gradeRange PK-5
enrollment 1002
gsRating 8
parentRating 4
city Canton
state GA
districtId 35
district Cherokee County School District
districtNCESId 1301110
address 6391 East Cherokee Drive, 
Canton, GA  30115
phone (770) 479-6200
fax None
website http://cherokeek12.net/averyes
ncesId 130111003309
lat 34.21437
lon -84.3946
overviewLink https://www.greatschools.org/georgia/canton/3945-Avery-Elementary-School/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/georgia/canton/3945-Avery-Elementary-School/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/georgia/canton/3945-Avery-Elementary-School/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/georgia/canton/3945-Avery-Elementary-School/?utm_source=GSAPI&utm_medium=referral
gsId 4563
name Avery Montessori Academy
type private
gradeRange K & ungraded
enrollment 20
city Waleska
state GA
addres

gsId 554
name Baker Elementary School
type public
gradeRange PK-5
enrollment 781
gsRating 9
parentRating 4
city Acworth
state GA
districtId 41
district Cobb County School District
districtNCESId 1301290
address 2361 Baker Road, 
Acworth, GA  30101
phone (770) 975-6629
fax (770) 975-6631
website http://www.cobbk12.org/Baker/
ncesId 130129002021
lat 34.057617
lon -84.62355
overviewLink https://www.greatschools.org/georgia/acworth/554-Baker-Elementary-School/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/georgia/acworth/554-Baker-Elementary-School/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/georgia/acworth/554-Baker-Elementary-School/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/georgia/acworth/554-Baker-Elementary-School/?utm_source=GSAPI&utm_medium=referral
gsId 2536
name Baker Middle School
type public
gradeRange 6-8
enrollment 566
gsRating 1
parentRating 2
city Columbus
stat

gsId 3682
name Barber Middle School
type public
gradeRange 6-8
enrollment 960
gsRating 7
parentRating 3
city Acworth
state GA
districtId 41
district Cobb County School District
districtNCESId 1301290
address 4222 Cantrell Road Northwest, 
Acworth, GA  30101
phone (770) 975-6764
fax (770) 529-0325
website http://www.cobbk12.org/Barber/
ncesId 130129003531
lat 34.05928
lon -84.65232
overviewLink https://www.greatschools.org/georgia/acworth/3682-Barber-Middle-School/?utm_source=GSAPI&utm_medium=referral
ratingsLink https://www.greatschools.org/georgia/acworth/3682-Barber-Middle-School/?utm_source=GSAPI&utm_medium=referral
reviewsLink https://www.greatschools.org/georgia/acworth/3682-Barber-Middle-School/?utm_source=GSAPI&utm_medium=referral#Reviews
schoolStatsLink https://www.greatschools.org/georgia/acworth/3682-Barber-Middle-School/?utm_source=GSAPI&utm_medium=referral
gsId 192
name Barden Elementary School
type public
gradeRange PK-5
enrollment 287
parentRating 3
city Macon
state GA
di

,address,city,fax,gradeRange,gsId,lat,lon,name,ncesId,overviewLink,...,schoolStatsLink,state,type,website,district,districtId,districtNCESId,enrollment,gsRating,parentRating
0,"838 W Main St, \nPrichard, AL 36610",Prichard,None,6-12,3708,30.736015,-88.09117,100 BLK Men of Greater Mobile's Phoenix Program,A0900001,https://www.greatschools.org/alabama/prichard/...,...,https://www.greatschools.org/alabama/prichard/...,AL,private,None,NaN,NaN,NaN,NaN,NaN,NaN
1,"626 North Daleville Ave, \nDaleville, AL 36322",Daleville,(334) 598-9006,K-6,468,31.310625,-85.71681,A M Windham Elementary School,010108001767,https://www.greatschools.org/alabama/daleville...,...,https://www.greatschools.org/alabama/daleville...,AL,public,http://www.daleville.k12.al.us/wes.html,Daleville City School District,45,0101080,539,5,3
2,"501 Beck St, \nAtmore, AL 36502",Atmore,(251) 368-0244,3-4,565,31.015285,-87.485054,A.C. Moore Elementary School,010135000482,https://www.greatschools.org/alabama/atmore/56...,...,https://www.greatschools.org/alabama/atmore/56...,AL,public,http://www.escambiak12.net,Escambia County School District,54,0101350,280,3,1
3,"300a North Trawick St, \nAbbeville, AL 36310",Abbeville,(334) 585-2012,1-12,2247,31.567364,-85.25238,Abbeville Alt School,010174002120,https://www.greatschools.org/alabama/abbeville...,...,https://www.greatschools.org/alabama/abbeville...,AL,public,http://www.henrycountyboe.org,Henry County School District,70,0101740,280,3,1
4,"M L Tillis Dr, \nAbbeville, AL 36310",Abbeville,None,PK-12,1572,31.576424,-85.26009,Abbeville Christian Academy,00001026,https://www.greatschools.org/alabama/abbeville...,...,https://www.greatschools.org/alabama/abbeville...,AL,private,https://acagenerals.org/,Henry County School District,70,0101740,280,3,5
